In [75]:
%config Completer.use_jedi = False
%matplotlib inline

/Users/ls/anaconda3/envs/xrs/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [76]:
import os
import pytest
import xarray as xr 
import numpy as np
from osgeo import gdal

import datashader as ds

from datashader.transfer_functions import shade

from xrspatial.terrain import generate_terrain
from xrspatial import hillshade, slope, aspect

import rasterio, rioxarray

from qgis.core import *


In [77]:

np.set_printoptions(threshold=1000000)
xr.set_options(display_width=80)
elevation = rioxarray.open_rasterio("/Users/ls/Downloads/elevation.tif")
elevation = elevation[0,:,:]
copy_data = elevation.data.copy()

attrs = {
 'STATISTICS_MAXIMUM': 248,
 'STATISTICS_MEAN': 155.52919753086,
 'STATISTICS_MINIMUM': 58,
 'STATISTICS_STDDEV': 18.411587144898,
 'STATISTICS_VALID_PERCENT': 100,
 'scale_factor': 1.0,
 'add_offset': 0.0,
 'grid_mapping': 'spatial_ref'
}

coord_y_np = np.asarray([ 89.66666667,  89.        ,  88.33333333,  87.66666667,
        87.        ,  86.33333333,  85.66666667,  85.        ,
        84.33333333,  83.66666667,  83.        ,  82.33333333,
        81.66666667,  81.        ,  80.33333333,  79.66666667,
        79.        ,  78.33333333,  77.66666667,  77.        ,
        76.33333333,  75.66666667,  75.        ,  74.33333333,
        73.66666667,  73.        ,  72.33333333,  71.66666667,
        71.        ,  70.33333333,  69.66666667,  69.        ,
        68.33333333,  67.66666667,  67.        ,  66.33333333,
        65.66666667,  65.        ,  64.33333333,  63.66666667,
        63.        ,  62.33333333,  61.66666667,  61.        ,
        60.33333333,  59.66666667,  59.        ,  58.33333333,
        57.66666667,  57.        ,  56.33333333,  55.66666667,
        55.        ,  54.33333333,  53.66666667,  53.        ,
        52.33333333,  51.66666667,  51.        ,  50.33333333,
        49.66666667,  49.        ,  48.33333333,  47.66666667,
        47.        ,  46.33333333,  45.66666667,  45.        ,
        44.33333333,  43.66666667,  43.        ,  42.33333333,
        41.66666667,  41.        ,  40.33333333,  39.66666667,
        39.        ,  38.33333333,  37.66666667,  37.        ,
        36.33333333,  35.66666667,  35.        ,  34.33333333,
        33.66666667,  33.        ,  32.33333333,  31.66666667,
        31.        ,  30.33333333,  29.66666667,  29.        ,
        28.33333333,  27.66666667,  27.        ,  26.33333333,
        25.66666667,  25.        ,  24.33333333,  23.66666667,
        23.        ,  22.33333333,  21.66666667,  21.        ,
        20.33333333,  19.66666667,  19.        ,  18.33333333,
        17.66666667,  17.        ,  16.33333333,  15.66666667,
        15.        ,  14.33333333,  13.66666667,  13.        ,
        12.33333333,  11.66666667,  11.        ,  10.33333333,
         9.66666667,   9.        ,   8.33333333,   7.66666667,
         7.        ,   6.33333333,   5.66666667,   5.        ,
         4.33333333,   3.66666667,   3.        ,   2.33333333,
         1.66666667,   1.        ,   0.33333333,  -0.33333333,
        -1.        ,  -1.66666667,  -2.33333333,  -3.        ,
        -3.66666667,  -4.33333333,  -5.        ,  -5.66666667,
        -6.33333333,  -7.        ,  -7.66666667,  -8.33333333,
        -9.        ,  -9.66666667, -10.33333333, -11.        ,
       -11.66666667, -12.33333333, -13.        , -13.66666667,
       -14.33333333, -15.        , -15.66666667, -16.33333333,
       -17.        , -17.66666667, -18.33333333, -19.        ,
       -19.66666667, -20.33333333, -21.        , -21.66666667,
       -22.33333333, -23.        , -23.66666667, -24.33333333,
       -25.        , -25.66666667, -26.33333333, -27.        ,
       -27.66666667, -28.33333333, -29.        , -29.66666667,
       -30.33333333, -31.        , -31.66666667, -32.33333333,
       -33.        , -33.66666667, -34.33333333, -35.        ,
       -35.66666667, -36.33333333, -37.        , -37.66666667,
       -38.33333333, -39.        , -39.66666667, -40.33333333,
       -41.        , -41.66666667, -42.33333333, -43.        ,
       -43.66666667, -44.33333333, -45.        , -45.66666667,
       -46.33333333, -47.        , -47.66666667, -48.33333333,
       -49.        , -49.66666667, -50.33333333, -51.        ,
       -51.66666667, -52.33333333, -53.        , -53.66666667,
       -54.33333333, -55.        , -55.66666667, -56.33333333,
       -57.        , -57.66666667, -58.33333333, -59.        ,
       -59.66666667, -60.33333333, -61.        , -61.66666667,
       -62.33333333, -63.        , -63.66666667, -64.33333333,
       -65.        , -65.66666667, -66.33333333, -67.        ,
       -67.66666667, -68.33333333, -69.        , -69.66666667,
       -70.33333333, -71.        , -71.66666667, -72.33333333,
       -73.        , -73.66666667, -74.33333333, -75.        ,
       -75.66666667, -76.33333333, -77.        , -77.66666667,
       -78.33333333, -79.        , -79.66666667, -80.33333333,
       -81.        , -81.66666667, -82.33333333, -83.        ,
       -83.66666667, -84.33333333, -85.        , -85.66666667,
       -86.33333333, -87.        , -87.66666667, -88.33333333,
       -89.        , -89.66666667], dtype=np.float64)

coord_x_np = np.asarray([-179.66666667, -179.        , -178.33333333, -177.66666667,
       -177.        , -176.33333333, -175.66666667, -175.        ,
       -174.33333333, -173.66666667, -173.        , -172.33333333,
       -171.66666667, -171.        , -170.33333333, -169.66666667,
       -169.        , -168.33333333, -167.66666667, -167.        ,
       -166.33333333, -165.66666667, -165.        , -164.33333333,
       -163.66666667, -163.        , -162.33333333, -161.66666667,
       -161.        , -160.33333333, -159.66666667, -159.        ,
       -158.33333333, -157.66666667, -157.        , -156.33333333,
       -155.66666667, -155.        , -154.33333333, -153.66666667,
       -153.        , -152.33333333, -151.66666667, -151.        ,
       -150.33333333, -149.66666667, -149.        , -148.33333333,
       -147.66666667, -147.        , -146.33333333, -145.66666667,
       -145.        , -144.33333333, -143.66666667, -143.        ,
       -142.33333333, -141.66666667, -141.        , -140.33333333,
       -139.66666667, -139.        , -138.33333333, -137.66666667,
       -137.        , -136.33333333, -135.66666667, -135.        ,
       -134.33333333, -133.66666667, -133.        , -132.33333333,
       -131.66666667, -131.        , -130.33333333, -129.66666667,
       -129.        , -128.33333333, -127.66666667, -127.        ,
       -126.33333333, -125.66666667, -125.        , -124.33333333,
       -123.66666667, -123.        , -122.33333333, -121.66666667,
       -121.        , -120.33333333, -119.66666667, -119.        ,
       -118.33333333, -117.66666667, -117.        , -116.33333333,
       -115.66666667, -115.        , -114.33333333, -113.66666667,
       -113.        , -112.33333333, -111.66666667, -111.        ,
       -110.33333333, -109.66666667, -109.        , -108.33333333,
       -107.66666667, -107.        , -106.33333333, -105.66666667,
       -105.        , -104.33333333, -103.66666667, -103.        ,
       -102.33333333, -101.66666667, -101.        , -100.33333333,
        -99.66666667,  -99.        ,  -98.33333333,  -97.66666667,
        -97.        ,  -96.33333333,  -95.66666667,  -95.        ,
        -94.33333333,  -93.66666667,  -93.        ,  -92.33333333,
        -91.66666667,  -91.        ,  -90.33333333,  -89.66666667,
        -89.        ,  -88.33333333,  -87.66666667,  -87.        ,
        -86.33333333,  -85.66666667,  -85.        ,  -84.33333333,
        -83.66666667,  -83.        ,  -82.33333333,  -81.66666667,
        -81.        ,  -80.33333333,  -79.66666667,  -79.        ,
        -78.33333333,  -77.66666667,  -77.        ,  -76.33333333,
        -75.66666667,  -75.        ,  -74.33333333,  -73.66666667,
        -73.        ,  -72.33333333,  -71.66666667,  -71.        ,
        -70.33333333,  -69.66666667,  -69.        ,  -68.33333333,
        -67.66666667,  -67.        ,  -66.33333333,  -65.66666667,
        -65.        ,  -64.33333333,  -63.66666667,  -63.        ,
        -62.33333333,  -61.66666667,  -61.        ,  -60.33333333,
        -59.66666667,  -59.        ,  -58.33333333,  -57.66666667,
        -57.        ,  -56.33333333,  -55.66666667,  -55.        ,
        -54.33333333,  -53.66666667,  -53.        ,  -52.33333333,
        -51.66666667,  -51.        ,  -50.33333333,  -49.66666667,
        -49.        ,  -48.33333333,  -47.66666667,  -47.        ,
        -46.33333333,  -45.66666667,  -45.        ,  -44.33333333,
        -43.66666667,  -43.        ,  -42.33333333,  -41.66666667,
        -41.        ,  -40.33333333,  -39.66666667,  -39.        ,
        -38.33333333,  -37.66666667,  -37.        ,  -36.33333333,
        -35.66666667,  -35.        ,  -34.33333333,  -33.66666667,
        -33.        ,  -32.33333333,  -31.66666667,  -31.        ,
        -30.33333333,  -29.66666667,  -29.        ,  -28.33333333,
        -27.66666667,  -27.        ,  -26.33333333,  -25.66666667,
        -25.        ,  -24.33333333,  -23.66666667,  -23.        ,
        -22.33333333,  -21.66666667,  -21.        ,  -20.33333333,
        -19.66666667,  -19.        ,  -18.33333333,  -17.66666667,
        -17.        ,  -16.33333333,  -15.66666667,  -15.        ,
        -14.33333333,  -13.66666667,  -13.        ,  -12.33333333,
        -11.66666667,  -11.        ,  -10.33333333,   -9.66666667,
         -9.        ,   -8.33333333,   -7.66666667,   -7.        ,
         -6.33333333,   -5.66666667,   -5.        ,   -4.33333333,
         -3.66666667,   -3.        ,   -2.33333333,   -1.66666667,
         -1.        ,   -0.33333333,    0.33333333,    1.        ,
          1.66666667,    2.33333333,    3.        ,    3.66666667,
          4.33333333,    5.        ,    5.66666667,    6.33333333,
          7.        ,    7.66666667,    8.33333333,    9.        ,
          9.66666667,   10.33333333,   11.        ,   11.66666667,
         12.33333333,   13.        ,   13.66666667,   14.33333333,
         15.        ,   15.66666667,   16.33333333,   17.        ,
         17.66666667,   18.33333333,   19.        ,   19.66666667,
         20.33333333,   21.        ,   21.66666667,   22.33333333,
         23.        ,   23.66666667,   24.33333333,   25.        ,
         25.66666667,   26.33333333,   27.        ,   27.66666667,
         28.33333333,   29.        ,   29.66666667,   30.33333333,
         31.        ,   31.66666667,   32.33333333,   33.        ,
         33.66666667,   34.33333333,   35.        ,   35.66666667,
         36.33333333,   37.        ,   37.66666667,   38.33333333,
         39.        ,   39.66666667,   40.33333333,   41.        ,
         41.66666667,   42.33333333,   43.        ,   43.66666667,
         44.33333333,   45.        ,   45.66666667,   46.33333333,
         47.        ,   47.66666667,   48.33333333,   49.        ,
         49.66666667,   50.33333333,   51.        ,   51.66666667,
         52.33333333,   53.        ,   53.66666667,   54.33333333,
         55.        ,   55.66666667,   56.33333333,   57.        ,
         57.66666667,   58.33333333,   59.        ,   59.66666667,
         60.33333333,   61.        ,   61.66666667,   62.33333333,
         63.        ,   63.66666667,   64.33333333,   65.        ,
         65.66666667,   66.33333333,   67.        ,   67.66666667,
         68.33333333,   69.        ,   69.66666667,   70.33333333,
         71.        ,   71.66666667,   72.33333333,   73.        ,
         73.66666667,   74.33333333,   75.        ,   75.66666667,
         76.33333333,   77.        ,   77.66666667,   78.33333333,
         79.        ,   79.66666667,   80.33333333,   81.        ,
         81.66666667,   82.33333333,   83.        ,   83.66666667,
         84.33333333,   85.        ,   85.66666667,   86.33333333,
         87.        ,   87.66666667,   88.33333333,   89.        ,
         89.66666667,   90.33333333,   91.        ,   91.66666667,
         92.33333333,   93.        ,   93.66666667,   94.33333333,
         95.        ,   95.66666667,   96.33333333,   97.        ,
         97.66666667,   98.33333333,   99.        ,   99.66666667,
        100.33333333,  101.        ,  101.66666667,  102.33333333,
        103.        ,  103.66666667,  104.33333333,  105.        ,
        105.66666667,  106.33333333,  107.        ,  107.66666667,
        108.33333333,  109.        ,  109.66666667,  110.33333333,
        111.        ,  111.66666667,  112.33333333,  113.        ,
        113.66666667,  114.33333333,  115.        ,  115.66666667,
        116.33333333,  117.        ,  117.66666667,  118.33333333,
        119.        ,  119.66666667,  120.33333333,  121.        ,
        121.66666667,  122.33333333,  123.        ,  123.66666667,
        124.33333333,  125.        ,  125.66666667,  126.33333333,
        127.        ,  127.66666667,  128.33333333,  129.        ,
        129.66666667,  130.33333333,  131.        ,  131.66666667,
        132.33333333,  133.        ,  133.66666667,  134.33333333,
        135.        ,  135.66666667,  136.33333333,  137.        ,
        137.66666667,  138.33333333,  139.        ,  139.66666667,
        140.33333333,  141.        ,  141.66666667,  142.33333333,
        143.        ,  143.66666667,  144.33333333,  145.        ,
        145.66666667,  146.33333333,  147.        ,  147.66666667,
        148.33333333,  149.        ,  149.66666667,  150.33333333,
        151.        ,  151.66666667,  152.33333333,  153.        ,
        153.66666667,  154.33333333,  155.        ,  155.66666667,
        156.33333333,  157.        ,  157.66666667,  158.33333333,
        159.        ,  159.66666667,  160.33333333,  161.        ,
        161.66666667,  162.33333333,  163.        ,  163.66666667,
        164.33333333,  165.        ,  165.66666667,  166.33333333,
        167.        ,  167.66666667,  168.33333333,  169.        ,
        169.66666667,  170.33333333,  171.        ,  171.66666667,
        172.33333333,  173.        ,  173.66666667,  174.33333333,
        175.        ,  175.66666667,  176.33333333,  177.        ,
        177.66666667,  178.33333333,  179.        ,  179.66666667] ,dtype=np.float64)

coord_y = coord_y_np
coord_x = coord_x_np

coords_spatial_ref_attrs = {
    'crs_wkt': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]',
    'semi_major_axis': 6378137.0,
    'semi_minor_axis': 6356752.314245179,
    'inverse_flattening': 298.257223563,
    'reference_ellipsoid_name': 'WGS 84',
    'longitude_of_prime_meridian': 0.0,
    'prime_meridian_name': 'Greenwich',
    'geographic_crs_name': 'WGS 84',
    'grid_mapping_name': 'latitude_longitude',
    'spatial_ref': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]', 'GeoTransform': '-180.0 0.6666666666666666 0.0 90.0 0.0 -0.6666666666666666'
}
elevation.data.dtype

dtype('uint8')

In [78]:
elevation_da = xr.DataArray(copy_data, attrs=attrs, dims=['y', 'x'])
elevation_da.coords['y'] = coord_y
elevation_da.coords['x'] = coord_x
elevation_da.coords['spatial_ref'] = xr.DataArray(0, attrs=coords_spatial_ref_attrs)
elevation_da.coords['band'] = np.asarray(1, dtype=np.int64)

/Users/ls/anaconda3/envs/xrs/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [94]:
np.set_printoptions(threshold=10000000)
xr.set_options(display_width=10000000)
qgis_slope = rioxarray.open_rasterio("/Users/ls/Downloads/OUTPUT.tif")[0,:,:]
qgis_slope.data = np.where(qgis_slope.data == -9999, np.nan, qgis_slope.data)
xrs_slope = slope(elevation_da)
# qgis_slope_vals = qgis_slope.values[1:-1, 1:-1]
# xrs_slope_vals = xrs_slope.values[1:-1, 1:-1]
np.isclose(qgis_slope.data, xrs_slope.data, equal_nan=True).all()
# print(qgis_slope.data[:, 34, 34])
# assert (np.isclose(qgis_slope.data, xrs_slope.data, equal_nan=True).all() | 
#         np.isnan(qgis_slope.data) & np.isnan(xrs_slope.data)).all()
qgis_slope.attrs

/Users/ls/anaconda3/envs/xrs/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'_FillValue': -9999.0,
 'scale_factor': 1.0,
 'add_offset': 0.0,
 'grid_mapping': 'spatial_ref'}

In [102]:
copy_slope_data = qgis_slope.data.copy()
coord_y_np = np.asarray([ 89.66666667,  89.        ,  88.33333333,  87.66666667,
        87.        ,  86.33333333,  85.66666667,  85.        ,
        84.33333333,  83.66666667,  83.        ,  82.33333333,
        81.66666667,  81.        ,  80.33333333,  79.66666667,
        79.        ,  78.33333333,  77.66666667,  77.        ,
        76.33333333,  75.66666667,  75.        ,  74.33333333,
        73.66666667,  73.        ,  72.33333333,  71.66666667,
        71.        ,  70.33333333,  69.66666667,  69.        ,
        68.33333333,  67.66666667,  67.        ,  66.33333333,
        65.66666667,  65.        ,  64.33333333,  63.66666667,
        63.        ,  62.33333333,  61.66666667,  61.        ,
        60.33333333,  59.66666667,  59.        ,  58.33333333,
        57.66666667,  57.        ,  56.33333333,  55.66666667,
        55.        ,  54.33333333,  53.66666667,  53.        ,
        52.33333333,  51.66666667,  51.        ,  50.33333333,
        49.66666667,  49.        ,  48.33333333,  47.66666667,
        47.        ,  46.33333333,  45.66666667,  45.        ,
        44.33333333,  43.66666667,  43.        ,  42.33333333,
        41.66666667,  41.        ,  40.33333333,  39.66666667,
        39.        ,  38.33333333,  37.66666667,  37.        ,
        36.33333333,  35.66666667,  35.        ,  34.33333333,
        33.66666667,  33.        ,  32.33333333,  31.66666667,
        31.        ,  30.33333333,  29.66666667,  29.        ,
        28.33333333,  27.66666667,  27.        ,  26.33333333,
        25.66666667,  25.        ,  24.33333333,  23.66666667,
        23.        ,  22.33333333,  21.66666667,  21.        ,
        20.33333333,  19.66666667,  19.        ,  18.33333333,
        17.66666667,  17.        ,  16.33333333,  15.66666667,
        15.        ,  14.33333333,  13.66666667,  13.        ,
        12.33333333,  11.66666667,  11.        ,  10.33333333,
         9.66666667,   9.        ,   8.33333333,   7.66666667,
         7.        ,   6.33333333,   5.66666667,   5.        ,
         4.33333333,   3.66666667,   3.        ,   2.33333333,
         1.66666667,   1.        ,   0.33333333,  -0.33333333,
        -1.        ,  -1.66666667,  -2.33333333,  -3.        ,
        -3.66666667,  -4.33333333,  -5.        ,  -5.66666667,
        -6.33333333,  -7.        ,  -7.66666667,  -8.33333333,
        -9.        ,  -9.66666667, -10.33333333, -11.        ,
       -11.66666667, -12.33333333, -13.        , -13.66666667,
       -14.33333333, -15.        , -15.66666667, -16.33333333,
       -17.        , -17.66666667, -18.33333333, -19.        ,
       -19.66666667, -20.33333333, -21.        , -21.66666667,
       -22.33333333, -23.        , -23.66666667, -24.33333333,
       -25.        , -25.66666667, -26.33333333, -27.        ,
       -27.66666667, -28.33333333, -29.        , -29.66666667,
       -30.33333333, -31.        , -31.66666667, -32.33333333,
       -33.        , -33.66666667, -34.33333333, -35.        ,
       -35.66666667, -36.33333333, -37.        , -37.66666667,
       -38.33333333, -39.        , -39.66666667, -40.33333333,
       -41.        , -41.66666667, -42.33333333, -43.        ,
       -43.66666667, -44.33333333, -45.        , -45.66666667,
       -46.33333333, -47.        , -47.66666667, -48.33333333,
       -49.        , -49.66666667, -50.33333333, -51.        ,
       -51.66666667, -52.33333333, -53.        , -53.66666667,
       -54.33333333, -55.        , -55.66666667, -56.33333333,
       -57.        , -57.66666667, -58.33333333, -59.        ,
       -59.66666667, -60.33333333, -61.        , -61.66666667,
       -62.33333333, -63.        , -63.66666667, -64.33333333,
       -65.        , -65.66666667, -66.33333333, -67.        ,
       -67.66666667, -68.33333333, -69.        , -69.66666667,
       -70.33333333, -71.        , -71.66666667, -72.33333333,
       -73.        , -73.66666667, -74.33333333, -75.        ,
       -75.66666667, -76.33333333, -77.        , -77.66666667,
       -78.33333333, -79.        , -79.66666667, -80.33333333,
       -81.        , -81.66666667, -82.33333333, -83.        ,
       -83.66666667, -84.33333333, -85.        , -85.66666667,
       -86.33333333, -87.        , -87.66666667, -88.33333333,
       -89.        , -89.66666667 ], dtype=np.float64)

coord_x_np = np.asarray([ -179.66666667, -179.        , -178.33333333, -177.66666667,
       -177.        , -176.33333333, -175.66666667, -175.        ,
       -174.33333333, -173.66666667, -173.        , -172.33333333,
       -171.66666667, -171.        , -170.33333333, -169.66666667,
       -169.        , -168.33333333, -167.66666667, -167.        ,
       -166.33333333, -165.66666667, -165.        , -164.33333333,
       -163.66666667, -163.        , -162.33333333, -161.66666667,
       -161.        , -160.33333333, -159.66666667, -159.        ,
       -158.33333333, -157.66666667, -157.        , -156.33333333,
       -155.66666667, -155.        , -154.33333333, -153.66666667,
       -153.        , -152.33333333, -151.66666667, -151.        ,
       -150.33333333, -149.66666667, -149.        , -148.33333333,
       -147.66666667, -147.        , -146.33333333, -145.66666667,
       -145.        , -144.33333333, -143.66666667, -143.        ,
       -142.33333333, -141.66666667, -141.        , -140.33333333,
       -139.66666667, -139.        , -138.33333333, -137.66666667,
       -137.        , -136.33333333, -135.66666667, -135.        ,
       -134.33333333, -133.66666667, -133.        , -132.33333333,
       -131.66666667, -131.        , -130.33333333, -129.66666667,
       -129.        , -128.33333333, -127.66666667, -127.        ,
       -126.33333333, -125.66666667, -125.        , -124.33333333,
       -123.66666667, -123.        , -122.33333333, -121.66666667,
       -121.        , -120.33333333, -119.66666667, -119.        ,
       -118.33333333, -117.66666667, -117.        , -116.33333333,
       -115.66666667, -115.        , -114.33333333, -113.66666667,
       -113.        , -112.33333333, -111.66666667, -111.        ,
       -110.33333333, -109.66666667, -109.        , -108.33333333,
       -107.66666667, -107.        , -106.33333333, -105.66666667,
       -105.        , -104.33333333, -103.66666667, -103.        ,
       -102.33333333, -101.66666667, -101.        , -100.33333333,
        -99.66666667,  -99.        ,  -98.33333333,  -97.66666667,
        -97.        ,  -96.33333333,  -95.66666667,  -95.        ,
        -94.33333333,  -93.66666667,  -93.        ,  -92.33333333,
        -91.66666667,  -91.        ,  -90.33333333,  -89.66666667,
        -89.        ,  -88.33333333,  -87.66666667,  -87.        ,
        -86.33333333,  -85.66666667,  -85.        ,  -84.33333333,
        -83.66666667,  -83.        ,  -82.33333333,  -81.66666667,
        -81.        ,  -80.33333333,  -79.66666667,  -79.        ,
        -78.33333333,  -77.66666667,  -77.        ,  -76.33333333,
        -75.66666667,  -75.        ,  -74.33333333,  -73.66666667,
        -73.        ,  -72.33333333,  -71.66666667,  -71.        ,
        -70.33333333,  -69.66666667,  -69.        ,  -68.33333333,
        -67.66666667,  -67.        ,  -66.33333333,  -65.66666667,
        -65.        ,  -64.33333333,  -63.66666667,  -63.        ,
        -62.33333333,  -61.66666667,  -61.        ,  -60.33333333,
        -59.66666667,  -59.        ,  -58.33333333,  -57.66666667,
        -57.        ,  -56.33333333,  -55.66666667,  -55.        ,
        -54.33333333,  -53.66666667,  -53.        ,  -52.33333333,
        -51.66666667,  -51.        ,  -50.33333333,  -49.66666667,
        -49.        ,  -48.33333333,  -47.66666667,  -47.        ,
        -46.33333333,  -45.66666667,  -45.        ,  -44.33333333,
        -43.66666667,  -43.        ,  -42.33333333,  -41.66666667,
        -41.        ,  -40.33333333,  -39.66666667,  -39.        ,
        -38.33333333,  -37.66666667,  -37.        ,  -36.33333333,
        -35.66666667,  -35.        ,  -34.33333333,  -33.66666667,
        -33.        ,  -32.33333333,  -31.66666667,  -31.        ,
        -30.33333333,  -29.66666667,  -29.        ,  -28.33333333,
        -27.66666667,  -27.        ,  -26.33333333,  -25.66666667,
        -25.        ,  -24.33333333,  -23.66666667,  -23.        ,
        -22.33333333,  -21.66666667,  -21.        ,  -20.33333333,
        -19.66666667,  -19.        ,  -18.33333333,  -17.66666667,
        -17.        ,  -16.33333333,  -15.66666667,  -15.        ,
        -14.33333333,  -13.66666667,  -13.        ,  -12.33333333,
        -11.66666667,  -11.        ,  -10.33333333,   -9.66666667,
         -9.        ,   -8.33333333,   -7.66666667,   -7.        ,
         -6.33333333,   -5.66666667,   -5.        ,   -4.33333333,
         -3.66666667,   -3.        ,   -2.33333333,   -1.66666667,
         -1.        ,   -0.33333333,    0.33333333,    1.        ,
          1.66666667,    2.33333333,    3.        ,    3.66666667,
          4.33333333,    5.        ,    5.66666667,    6.33333333,
          7.        ,    7.66666667,    8.33333333,    9.        ,
          9.66666667,   10.33333333,   11.        ,   11.66666667,
         12.33333333,   13.        ,   13.66666667,   14.33333333,
         15.        ,   15.66666667,   16.33333333,   17.        ,
         17.66666667,   18.33333333,   19.        ,   19.66666667,
         20.33333333,   21.        ,   21.66666667,   22.33333333,
         23.        ,   23.66666667,   24.33333333,   25.        ,
         25.66666667,   26.33333333,   27.        ,   27.66666667,
         28.33333333,   29.        ,   29.66666667,   30.33333333,
         31.        ,   31.66666667,   32.33333333,   33.        ,
         33.66666667,   34.33333333,   35.        ,   35.66666667,
         36.33333333,   37.        ,   37.66666667,   38.33333333,
         39.        ,   39.66666667,   40.33333333,   41.        ,
         41.66666667,   42.33333333,   43.        ,   43.66666667,
         44.33333333,   45.        ,   45.66666667,   46.33333333,
         47.        ,   47.66666667,   48.33333333,   49.        ,
         49.66666667,   50.33333333,   51.        ,   51.66666667,
         52.33333333,   53.        ,   53.66666667,   54.33333333,
         55.        ,   55.66666667,   56.33333333,   57.        ,
         57.66666667,   58.33333333,   59.        ,   59.66666667,
         60.33333333,   61.        ,   61.66666667,   62.33333333,
         63.        ,   63.66666667,   64.33333333,   65.        ,
         65.66666667,   66.33333333,   67.        ,   67.66666667,
         68.33333333,   69.        ,   69.66666667,   70.33333333,
         71.        ,   71.66666667,   72.33333333,   73.        ,
         73.66666667,   74.33333333,   75.        ,   75.66666667,
         76.33333333,   77.        ,   77.66666667,   78.33333333,
         79.        ,   79.66666667,   80.33333333,   81.        ,
         81.66666667,   82.33333333,   83.        ,   83.66666667,
         84.33333333,   85.        ,   85.66666667,   86.33333333,
         87.        ,   87.66666667,   88.33333333,   89.        ,
         89.66666667,   90.33333333,   91.        ,   91.66666667,
         92.33333333,   93.        ,   93.66666667,   94.33333333,
         95.        ,   95.66666667,   96.33333333,   97.        ,
         97.66666667,   98.33333333,   99.        ,   99.66666667,
        100.33333333,  101.        ,  101.66666667,  102.33333333,
        103.        ,  103.66666667,  104.33333333,  105.        ,
        105.66666667,  106.33333333,  107.        ,  107.66666667,
        108.33333333,  109.        ,  109.66666667,  110.33333333,
        111.        ,  111.66666667,  112.33333333,  113.        ,
        113.66666667,  114.33333333,  115.        ,  115.66666667,
        116.33333333,  117.        ,  117.66666667,  118.33333333,
        119.        ,  119.66666667,  120.33333333,  121.        ,
        121.66666667,  122.33333333,  123.        ,  123.66666667,
        124.33333333,  125.        ,  125.66666667,  126.33333333,
        127.        ,  127.66666667,  128.33333333,  129.        ,
        129.66666667,  130.33333333,  131.        ,  131.66666667,
        132.33333333,  133.        ,  133.66666667,  134.33333333,
        135.        ,  135.66666667,  136.33333333,  137.        ,
        137.66666667,  138.33333333,  139.        ,  139.66666667,
        140.33333333,  141.        ,  141.66666667,  142.33333333,
        143.        ,  143.66666667,  144.33333333,  145.        ,
        145.66666667,  146.33333333,  147.        ,  147.66666667,
        148.33333333,  149.        ,  149.66666667,  150.33333333,
        151.        ,  151.66666667,  152.33333333,  153.        ,
        153.66666667,  154.33333333,  155.        ,  155.66666667,
        156.33333333,  157.        ,  157.66666667,  158.33333333,
        159.        ,  159.66666667,  160.33333333,  161.        ,
        161.66666667,  162.33333333,  163.        ,  163.66666667,
        164.33333333,  165.        ,  165.66666667,  166.33333333,
        167.        ,  167.66666667,  168.33333333,  169.        ,
        169.66666667,  170.33333333,  171.        ,  171.66666667,
        172.33333333,  173.        ,  173.66666667,  174.33333333,
        175.        ,  175.66666667,  176.33333333,  177.        ,
        177.66666667,  178.33333333,  179.        ,  179.66666667 ], dtype=np.float64)


coords_spatial_ref_attrs = {
    'crs_wkt': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]', 
    'semi_major_axis': 6378137.0, 
    'semi_minor_axis': 6356752.314245179, 
    'inverse_flattening': 298.257223563, 
    'reference_ellipsoid_name': 'WGS 84', 
    'longitude_of_prime_meridian': 0.0, 
    'prime_meridian_name': 'Greenwich', 
    'geographic_crs_name': 'WGS 84', 
    'grid_mapping_name': 'latitude_longitude', 
    'spatial_ref': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]', 
    'GeoTransform': '-180.0 0.6666666666666666 0.0 90.0 0.0 -0.6666666666666666'
}

attrs = {
'_FillValue': -9999.0,
'scale_factor': 1.0,
'add_offset': 0.0,
'grid_mapping': 'spatial_ref'
}

qgis_slope_da = xr.DataArray(copy_slope_data, attrs=attrs, dims=['y', 'x'])
qgis_slope_da.coords['y'] = coord_y_np
qgis_slope_da.coords['x'] = coord_x_np
qgis_slope_da.coords['spatial_ref'] = xr.DataArray(0, attrs=coords_spatial_ref_attrs)
qgis_slope_da.coords['band'] = np.asarray(1, dtype=np.int64)


<xarray.DataArray 'band' ()>
array(True)
Coordinates:
    band         int64 1
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(True)
Coordinates:
    band         int64 1
    spatial_ref  int64 0
<xarray.DataArray 'x' ()>
array(True)
Coordinates:
    band         int64 1
    spatial_ref  int64 0
<xarray.DataArray 'spatial_ref' ()>
array(True)
Coordinates:
    band         int64 1
    spatial_ref  int64 0


/Users/ls/anaconda3/envs/xrs/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [118]:
np.set_printoptions(threshold=10)
qgis_aspect = rioxarray.open_rasterio("/Users/ls/Downloads/qgis_aspect_b.tif")[0, :, :]
qgis_aspect.data = np.where(qgis_aspect.data == -9999, np.nan, qgis_aspect.data)
xrs_aspect = aspect(elevation)
xrs_aspect.data = np.where(xrs_aspect.data == -1, np.nan, xrs_aspect.data)
xrs_aspect.data = xrs_aspect.data.astype(np.float32)
np.isclose(xrs_aspect.data, qgis_aspect.data, equal_nan=True, atol=1e-7).all()

True

In [117]:
np.set_printoptions(threshold=100000000)
print(repr(np.where(qgis_aspect.data == np.nan, type(qgis_aspect.data, ), qgis_aspect.data)))

/Users/ls/anaconda3/envs/xrs/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan

In [74]:
elevation_m = rioxarray.open_rasterio("/Users/ls/Downloads/elevation_epsg3857.tif")[0, :, :]
qhs_hillshade = rioxarray.open_rasterio("/Users/ls/Downloads/qhs_hscc.tif")[0, :, :]
xrs_hillshade = hillshade(elevation_m, azimuth=315, angle_altitude=45)
qhs_hillshade.data[1:-1,1:-1], xrs_hillshade.data[1:-1,1:-1] * 255

/Users/ls/anaconda3/envs/xrs/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
 array([[217.6561146 , 146.1719427 , 217.6561146 , ..., 217.6561146 ,
         146.1719427 , 217.6561146 ],
        [146.1719427 , 179.55165703, 236.3280573 , ..., 146.1719427 ,
         146.1719427 , 236.3280573 ],
        [179.55165703, 253.16381636, 236.3280573 , ..., 146.1719427 ,
         236.3280573 , 179.55165703],
        ...,
        [236.3280573 ,  66.99268908,  72.17415918, ..., 236.3280573 ,
         199.15054301, 199.90448957],
        [179.55165703,  66.65137604,  73.03724782, ..., 236.3280573 ,
         199.15054301, 199.90448957],
        [146.1719427 ,  72.17415918,  73.03724782, ..., 236.3280573 ,
         199.15054301, 199.90448957]]))

/Users/ls/anaconda3/envs/xrs/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.0

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



aspect,
bump,
classify,
convolution,
curvature,
esri,
focal, 
hillshade,
multispectral,
pathfinding,
perlin,
proximity,
slope,
terrain,
tiles,
utils,
viewshed,
zonal